In [2]:
import pandas as pd
import numpy as np
import re

fertility = pd.read_csv('project_data/합계출산.csv', encoding='utf-8')
fertility.columns = ['합계출산;'+i for i in fertility.columns]
hdi = pd.read_csv('project_data/인간개발지수.csv',encoding='utf-8', header=[0,1])
hdi.columns = [';'.join(i) for i in hdi.columns]
hdi['국가별;국가별'] = hdi['국가별;국가별'].apply(lambda x: x.strip())
gdp = pd.read_csv('project_data/국민소득.csv', encoding='utf-8', header=[0,1])
gdp.columns = [';'.join(i) for i in gdp.columns]
urban = pd.read_csv('project_data/도시화율.csv', encoding='utf-8')
urban.columns = ['도시화율;'+i for i in urban.columns]
pop = pd.read_csv('project_data/인구_동향.csv', encoding='utf-8', header=[0,1])
pop.columns = [';'.join(i) for i in pop.columns]

merged_df = gdp.copy()
merged_df
for file in (fertility, urban, hdi, pop):
    try:
        merged_df = merged_df.merge(file,
                              'outer',
                              left_on = '국가;국가',
                              right_on = '국가별;국가별',
                              )
    except:
        merged_df = merged_df.merge(file,
                              'outer',
                              left_on = '국가;국가',
                              right_on = file.columns[file.columns.str.endswith('국가별')][0],
                              )
nation_column = merged_df[merged_df.columns[merged_df.columns.str.contains('국가(별|;)')]]
merged_df = merged_df.drop(
    columns = merged_df.columns[merged_df.columns.str.contains('국가(별|;)')])
merged_df['국가'] = nation_column.apply(lambda x: x[x.isna()==False].iloc[0], axis=1)

def all_na(df_col):
    not_na = df_col[df_col.isna()==False]
    if len(not_na)>0:
        return not_na.iloc[0]
    else:
        return np.nan
duplicate = merged_df['국가'].value_counts()[merged_df['국가'].value_counts()>1].index
rm_na_df = merged_df[merged_df['국가'].isin(duplicate)].\
                                groupby('국가').apply(lambda x: x.apply(all_na))
merged_df = merged_df.drop(index = merged_df[merged_df['국가'].isin(duplicate)].index)
final_data = pd.concat([merged_df,rm_na_df], axis=0)
final_data.set_index(['국가'], inplace=True)
final_data.replace('-',np.NaN, inplace = True)
final_data.to_csv('210809자료합친파일.csv', encoding="utf-8-sig")

C:\Users\song\anaconda3\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [3]:
for col in final_data.columns:
    final_data[col] = final_data[col].apply(lambda data : data if pd.isna(data) else np.NaN if data =='-' else re.findall('[0-9.]+',str(data))[0])
final_data = final_data.astype(float)
final_data

,2018;국내총생산(GDP $),2019;국내총생산(GDP $),합계출산;2015,합계출산;2020,도시화율;2020,평균교육기간(년);2018,1인당 GNI(구매력환산기준 2011) (달러);2018,2020;연간 인구증가율,2020;0~14세 인구 구성비,2020;15~64세 인구 구성비,2020;65세이상 인구 구성비
국가,,,,,,,,,,,
아프가니스탄,1.835388e+10,1.929110e+10,5.45,4.56,26.0,3.9,1.0,2.304,41.823,55.528,2.649
알바니아,1.514702e+10,1.528661e+10,1.71,1.62,62.1,10.1,12.0,0.578,17.236,68.060,14.705
알제리,1.754150e+11,1.711578e+11,2.96,3.05,73.7,8.0,13.0,1.837,30.784,62.474,6.743
미국령사모아,6.410000e+08,6.380000e+08,NaN,NaN,NaN,NaN,NaN,0.208,NaN,NaN,NaN
안도라,3.218316e+09,3.155065e+09,NaN,NaN,87.9,10.2,48.0,0.154,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
에리트레아,NaN,NaN,4.35,4.10,41.3,3.9,1.0,NaN,NaN,NaN,NaN
오세아니아,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
유럽,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
